In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

In [36]:
# Listando os arquivos na pasta 'input' com extensão .csv
input_folder = "input"
csv_files = [file for file in os.listdir(input_folder) if file.endswith(".csv")]

# Listando os arquivos
arquivos_desejados = [
    "SINASC_RO_2019_MAR.csv",
    "SINASC_RO_2019_ABR.csv",
    "SINASC_RO_2019_MAI.csv",
    "SINASC_RO_2019_JUN.csv",
    "SINASC_RO_2019_DEZ.csv"
]

# Verificando se os arquivos desejados existem e carregando cada um deles em um DataFrame
dataframes = {}
for arquivo in arquivos_desejados:
    if arquivo in csv_files:
        caminho_arquivo = os.path.join(input_folder, arquivo)
        dataframes[arquivo] = pd.read_csv(caminho_arquivo)
    else:
        print(f"O arquivo '{arquivo}' não foi encontrado na pasta 'input'.")

# Função para extrair o nome do mês a partir do nome do arquivo
def extrair_mes(nome_arquivo):
    mes = re.search(r'_([A-Z]+)\.csv', nome_arquivo)
    if mes:
        return mes.group(1)
    else:
        return None

# Criar a pasta 'resultado_grafico' caso não exista
pasta_resultado = "resultado_grafico"
if not os.path.exists(pasta_resultado):
    os.makedirs(pasta_resultado)

# Dicionário para converter nomes dos meses para português
meses_em_portugues = {
    "JAN": "Janeiro",
    "FEV": "Fevereiro",
    "MAR": "Março",
    "ABR": "Abril",
    "MAI": "Maio",
    "JUN": "Junho",
    "JUL": "Julho",
    "AGO": "Agosto",
    "SET": "Setembro",
    "OUT": "Outubro",
    "NOV": "Novembro",
    "DEZ": "Dezembro"
}

# Criando os gráficos de nascimentos por mês e salvando em PNG
total_nascimentos_por_mes = pd.Series(dtype='int64')
for idx, (arquivo, df) in enumerate(dataframes.items(), start=1):
    mes = extrair_mes(arquivo)
    if mes:
        df['DTNASC'] = pd.to_datetime(df['DTNASC'])
        df['DIA'] = df['DTNASC'].dt.day

        # Definindo o tamanho do gráfico
        plt.figure(figsize=(12, 6))

        nascimentos_por_dia = df.groupby('DIA').size()
        total_dias_mes = df['DIA'].max()
        intervalo = 2 if total_dias_mes <= 10 else 3  # Definindo intervalo de 2 ou 3 dependendo do total de dias

        nascimentos_por_dia.plot(kind='line', title=f"Nascimentos em {meses_em_portugues[mes]}", xlabel="Dia", ylabel="Quantidade")
        plt.xticks(range(1, total_dias_mes + 1, intervalo))  # Definindo o intervalo do eixo x

        # Adicionando grid
        plt.grid(True)

        # Salvando  gráfico como PNG
        nome_arquivo = os.path.join(pasta_resultado, f"{idx} - {meses_em_portugues[mes]}.png")
        plt.savefig(nome_arquivo)

        plt.close()  

        # Somando total de nascimentos por mês
        total_nascimentos_por_mes[mes] = df.shape[0]

    else:
        print(f"Não foi possível extrair o mês do arquivo '{arquivo}'.")

# Gerando  gráfico de quantidade de nascimentos por mês
cores_personalizadas = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'] 
plt.figure(figsize=(12, 6))
ax = total_nascimentos_por_mes.plot(kind='bar', title="Quantidade de Nascimentos por Mês", xlabel="Mês", ylabel="Quantidade", color=cores_personalizadas)
plt.xticks(rotation=45) 

# Adicionando valores nas barras
for i, v in enumerate(total_nascimentos_por_mes):
    ax.text(i, v, str(v), ha='center', va='bottom')

# Salvando gráfico adicional com a soma total de cada mês
nome_arquivo_geral = os.path.join(pasta_resultado, "Geral.png")
plt.savefig(nome_arquivo_geral)

plt.close()


**Descrição de como deixar o script mais automatizado:**

* Criar uma função para carregador os dados dos arquivos CSV em uma determinada pasta e transformá-los em DataFrames, depois criar uma função para gerar o gráfico de linha para cada mês específico retirado de cada arquivo contido na pasta e adicioná-los a uma pasta criada automaticamente ou previamente criada, desta forma seria necessário somente adicionar o arquivo a pasta e rodar o código novamente, o código iria ignorar os arquivos já existentes e já criados e adicionaria somente os arquivos considerados novos.